In [1]:
#!pip install praw transformers torch
import praw
import pandas as pd
import numpy as np
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
user_agent = "Brand sentiment analysis"
reddit = praw.Reddit(
  client_id="ujQpmAa-rxhI6tgE246E4w",
  client_secret="BnhybdI_fallhH64Us6vutIfsXP21Q",
  user_agent=user_agent
)

In [3]:
def fetch_reddit_posts(subreddit, query, limit=100):
    subreddit = reddit.subreddit(subreddit)
    posts = []
    for post in subreddit.search(query, limit=limit):
        if post.selftext and len(post.selftext.split()) > 6 and len(post.selftext.split()) <= 512:  # This checks if the body (selftext) is not empty
          posts.append([post.created_utc, post.title, post.selftext])
    df = pd.DataFrame(posts, columns=["Date", "Title", "Body"])
    df['Date'] = pd.to_datetime(df['Date'], unit='s')
    return df


In [5]:
df_neutrogena = fetch_reddit_posts('all', 'Neutrogena', 100)
# df_gemini = fetch_reddit_posts('all', 'Gemini AI', 100)
# df_copilot = fetch_reddit_posts('all', 'Copilot AI', 100)

In [6]:
def clean_text(text):
  text = re.sub(r'[^\x00-\x7F]+', ' ', text)
  text = re.sub(r'\s+', ' ', text)
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'[^\w\s]', '', text)
  return text.lower()

df_neutrogena['Body'] = df_neutrogena['Body'].apply(clean_text)
df_neutrogena['Title'] = df_neutrogena['Title'].apply(clean_text)

# df_chatgpt['Body'] = df_chatgpt['Body'].apply(clean_text)
# df_gemini['Body'] = df_gemini['Body'].apply(clean_text)
# df_copilot['Body'] = df_copilot['Body'].apply(clean_text)
# df_chatgpt['Title'] = df_chatgpt['Title'].apply(clean_text)
# df_gemini['Title'] = df_gemini['Title'].apply(clean_text)
# df_copilot['Title'] = df_copilot['Title'].apply(clean_text)

In [8]:
df_neutrogena.head()

,Date,Title,Body
0,2021-08-08 00:28:19,misc your thoughts on neutrogena,can i just say that i think neutrogena is the ...
1,2024-01-19 15:57:01,product question thoughts on the neutrogena hy...,i just bought this for the sole purpose of usi...
2,2023-10-16 17:00:55,neutrogena spf 50 is now pa finally,has neutrogena revamped their sunscreen to pa ...
3,2023-09-24 03:28:36,product question what s up with neutrogena hyd...,a few months ago i picked up a 2pack of neutro...
4,2024-01-23 03:13:25,hate the new neutrogena ultra sheer sunscreen ...,i have been using the old neutrogena ultra she...


In [11]:
df_neutrogena['source'] = 'Reddit'
df_neutrogena['language'] = 'English'

In [13]:
df_neutrogena.to_csv('neutrogena_reddit_output1.csv')